In [16]:
import pandas as pd

In [ ]:
# https://arxiv.org/pdf/2004.04906.pdf 
# KEY PAPER

# two datasets we want to implement:
# - SQUAD 2.0
# - NQ

In [17]:
import numpy as np
import torch
import json
from nltk.tokenize.treebank import TreebankWordDetokenizer
from tqdm import tqdm


def squad_processing(f):
    with open(f) as file:
        data = json.load(file)
    for par in data['data']:
        title = " ".join((par['title'].split("_")))
        for elem in par['paragraphs']:
            context = elem['context']
            if title.lower() not in context[:100].lower():
                context = title + " " + context
                elem['context'] = context
                for chunk in elem['qas']:
                    for ans in chunk['answers']:
                        ans['answer_start'] += len(title)+1
                print(elem)
                        

def NQ_processing(f):
    print('STARTING PROCESSING')
    file_data = []
    with open(f, 'r') as file:
        json_l = list(file)
        for json_str in tqdm(json_l):
            result = json.loads(json_str)
            file_data.append(result)
        
    docs = []
    questions = []
    
    # collecting the examples themselves
    x = file_data
    for k in tqdm(range(len(x))):
        #print(x[k].keys())
        doc_tokens = [x[k]['document_tokens'][i]['token'] for i in range(len(x[k]['document_tokens'])) if x[k]['document_tokens'][i]['html_token'] == False] 
        
        """
        for i in range(len(x[k]['document_tokens'])):
            if x[k]['document_tokens'][i]['html_token'] == False:
                doc_tokens.append(x[k]['document_tokens'][i]['token'])
        answers_text = []
        answers_ranges = []
        for j in range(len(x[k]['long_answer_candidates'])):
            start_token = x[k]['long_answer_candidates'][j]['start_token']
            end_token = x[k]['long_answer_candidates'][j]['end_token']
            answers_ranges.append([start_token, end_token])
            answers_text.append(doc_tokens[start_token: end_token])
        """
        docs.append(TreebankWordDetokenizer().detokenize(doc_tokens))
        questions.append(TreebankWordDetokenizer().detokenize(x[k]['question_tokens']))
        
    examples = {'doc':docs, 'question': questions}    
        
    # writing to a pandas dataframe 
    with open('NQ_clean_sample_full.json', 'w') as f:
        json.dump(examples, f)

    return examples


In [13]:
#x = NQ_processing("NQ/v1.0-simplified_simplified-nq-train.jsonl")
x = NQ_processing("NQ/v1.0_sample_nq-dev-sample.jsonl")

STARTING PROCESSING


100%|██████████| 200/200 [00:03<00:00, 51.84it/s]


In [15]:
from rank_bm25 import BM25Okapi
from tqdm import tqdm
from nltk.tokenize import word_tokenize

# find the indices of the k max elements
def kmax(k, nums, blocked):
    ix = []
    while len(ix) < k:
        w = max(nums)
        w_ix = nums.index(w)
        nums[w_ix] = float('-inf')
        ix.append(w_ix)
        if w_ix == blocked:
            ix.remove(w)
    return ix

def sample(j, nums, blocked):
    rand = random.sample(list(range(len(nums))), 30)
    for i in rand:
        if rand[i] in blocked:
            rand.remove(rand[i])
    return rand[0:20]
        
        
# CREATING OUR DATASET
#   DOC INDEX = CORRECT QUESTION INDEX, [HARD NEGATIVES INDICES], [EASY NEGATIVES INDICES]

k_hard = 10
j_rand = 20

score = 0
for i in tqdm(range(len(x['doc']))):
    
    tokenized_corpus = [word_tokenize(a) for a in x['doc']]
    tokenized_query = word_tokenize(x['questions'][i])
    bm25 = BM25Okapi(tokenized_corpus)
    doc_scores = bm25.get_scores(tokenized_query)
    
    hard_ix = kmax(k_hard, doc_scores, i)
    rand_ix = sample(j_rand, doc_scores, hard_ix + [i])
    
    answers = [i] + hard_ix + rand_ix
    print(answers)




  0%|          | 0/200 [00:06<?, ?it/s]


KeyError: 'questions'

In [48]:
x = NQ_processing("NQ/v1.0_sample_nq-dev-sample.jsonl")

{'annotations': [{'annotation_id': 14637775166913162648, 'long_answer': {'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}, 'short_answers': [], 'yes_no_answer': 'NONE'}, {'annotation_id': 15029374186306603138, 'long_answer': {'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}, 'short_answers': [], 'yes_no_answer': 'NONE'}, {'annotation_id': 15776890019256103141, 'long_answer': {'candidate_index': 18, 'end_byte': 48136, 'end_token': 480, 'start_byte': 47609, 'start_token': 388}, 'short_answers': [{'end_byte': 47736, 'end_token': 410, 'start_byte': 47716, 'start_token': 408}], 'yes_no_answer': 'NONE'}, {'annotation_id': 5789168769607885495, 'long_answer': {'candidate_index': -1, 'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1}, 'short_answers': [], 'yes_no_answer': 'NONE'}, {'annotation_id': 8396539049880339243, 'long_answer': {'candidate_index': -1, 'end_byte': -1, 'end_token': -1,

In [ ]:
def easy_negatives():
    return 0
    
def hard_negatives():
    return 0

    


In [ ]:
x[0][]

In [41]:
from bs4 import BeautifulSoup
import re
print(x[0])
"""

soup = BeautifulSoup(x[0]['document_html'], 'html.parser')
line = soup.get_text().replace('\n','').replace('\t','').replace('\xa0','').replace('\u200b','')
line = re.sub(r"</?\[\d+>", "", line)
line = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", "",line)
#line.split("(?<!(^|[A-Z]))(?=[A-Z])|(?<!^)(?=[A-Z][a-z])")
#line
#x = "camelValue"
re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', line)

#f.split('Whitespace characters')[1]"""

c


'\n\nsoup = BeautifulSoup(x[0][\'document_html\'], \'html.parser\')\nline = soup.get_text().replace(\'\n\',\'\').replace(\'\t\',\'\').replace(\'\xa0\',\'\').replace(\'\u200b\',\'\')\nline = re.sub(r"</?\\[\\d+>", "", line)\nline = re.sub(r"https?:\\/\\/(www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\x08([-a-zA-Z0-9()@:%_\\+.~#?&//=]*)", "",line)\n#line.split("(?<!(^|[A-Z]))(?=[A-Z])|(?<!^)(?=[A-Z][a-z])")\n#line\n#x = "camelValue"\nre.findall(r\'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))\', line)\n\n#f.split(\'Whitespace characters\')[1]'

In [42]:
from nltk.tokenize import sent_tokenize, word_tokenize
st = sent_tokenize(f)